# 🐧 The Evolution of Puffin's Population Regarding Climate Change

Welcome to the main Jupyter Notebook of our _Python project for Data Science_ ! 
Studying at the engineering school ENSAE, Institut Polytechnique de Paris, this project is part of the 2nd year course, under the supervision of Lino Galiana (Insee) and Romain Avouac (Insee). 

We share a passion for a clumsy but reaaaally cute seabird that is very common in Iceland : **the Atlantic puffin**. Unluckily, its demographic trend is downward. Well, it's not all by chance. Various high-qualited scientific studies have been carried out over the years to demonstrate the challenges facing this seabird species. Its biggest enemy to date is **climate change**, which is affecting its feeding (fish), reproduction and nesting. 

With this notebook, we set out to (re)demonstrate the causal link between global warming and the decline in the Atlantic puffin population. **The aim at the end of this page is to establish medium-term predictions for the evolution of this species, taking into account the different climate scenarios envisaged. Graphical visualization tools will support our modeling results.** 

Are you ready to see the disastrous consequences of climate change on such adorable birds? Let's get started! 
🐧🐧🐧🐧🐧🐧

## 💻 0. Setting up the work environment 

- data access through the cloud MinIO Client (files are in the folder 'diffusion')
- required packages 
- organizated environment 

In [19]:
import pandas as pd
import s3fs
import os
import xarray as xr

# Geographical vizualisation packages
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
import matplotlib
import contextily as ctx
import folium
from folium import LayerControl
from IPython.display import display
import webbrowser

# Creation of two folders to centralize data and results
data_dir = "./data"  
os.makedirs(data_dir, exist_ok=True) 
results_dir = "./results"
os.makedirs(results_dir, exist_ok=True)


# Access information to the cloud MinIO Client (Eve's bucket)
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})
MY_BUCKET = "esam"

## 🗺️ xx : Map visualization of Atlantic puffin distribution worldwide 

The Atlantic puffin is a migratory bird of the North Atlantic and Arctic polar zones. BirdLife International's data enable us to visualize the presence of these birds across the globe. 

### 2.1 MinIO Client cloud data retrieval and overview

_Nota bene : Having encountered difficulties working with files directly in S3 streams (shapefile, .nc, etc.), it is preferable to download files directly from the MinIO Client locally, to ensure optimal operation of the pandas and geopandas ecosystems, which is what the following program does._ 

In [3]:
# Creation of a subfolder to store Shapefile files and downloading of needed files
local_shapefile_subfolder = os.path.join(data_dir, "local_shapefile_files")
os.makedirs(local_shapefile_subfolder, exist_ok=True)

shapefile_elements = ["F_arctica.shp", "F_arctica.shx", "F_arctica.dbf"]
for element in shapefile_elements:
    remote_path = f"{MY_BUCKET}/diffusion/puffin_data/{element}"
    local_path = os.path.join(local_shapefile_subfolder, element)
    with fs.open(remote_path, "rb") as remote_file:
        with open(local_path, "wb") as local_file:
            local_file.write(remote_file.read())

# Lecture of the main shapefile file
local_shapefile_path = os.path.join(local_shapefile_subfolder, "F_arctica.shp")
gdf = gpd.read_file(local_shapefile_path)

### 2.2 Preparing files for graphical display

Geographic data and geometries must be correctly parameterized to be correctly displayed on a map.
The 'Coordinate Reference System' (CRS) EPSG:4326 displays geographic coordinates based on longitude and latitude. Its main uses are : 
- satellite data 
- global cartography 
- geographic data reference system

In [8]:
# Correction of the CRS to obtain the correct lecture of geographic coordinates
if gdf.crs != "EPSG:4326":
    print("The initial CRS is :", gdf.crs)
    gdf = gdf.set_crs(epsg=4326)
    print("The CRS has been redefined as EPSG:4326.")

# Simplification of the geometries
#if len(gdf) > 1000:  # Par exemple, pour les fichiers volumineux
gdf["geometry"] = gdf["geometry"].simplify(tolerance=0.01)
print("Geometries have been simplified.")

# Conversion to 2D geometries
def convert_to_2d(geom):
    if geom is not None and geom.has_z:
        return geom.simplify(0)  # Suppression de la dimension Z
    return geom

gdf["geometry"] = gdf["geometry"].apply(convert_to_2d)

# Checking of validated geometries
invalid_count = (~gdf.is_valid).sum()
print(f"Number of invalided geometries before correction : {invalid_count}")

# Correction of invalided geometries
if invalid_count > 0:
    gdf["geometry"] = gdf["geometry"].buffer(0)

Geometries have been simplified.
Number of invalided geometries before correction : 0


### 2.3 Creation of a reactive folium map to visualize the worldwide presence of Atlantic puffins.

The Atlantic puffin lives on the high seas all year round, but returns to land when it breeds. This map shows the areas of non-breeding and breeding of the species, using reference data from BirdLife International.

The detailed puffin observation databases we use are established in breeding areas. 

In [ ]:
# Definition of both layers to display on the map
breeding_zones = gdf[gdf["seasonal"] == 2]
non_breeding_zones = gdf[gdf["seasonal"] == 3]


# Creation of the Folium map and its layers 
m = folium.Map(location=[55, 0], zoom_start=3, tiles="CartoDB Positron")

def create_folium_layer(gdf, name, color):
    """Create a GeoJSON layer for a given GeoDataFrame."""
    return folium.GeoJson(
        gdf,
        name=name,
        tooltip=folium.GeoJsonTooltip(
            fields=["sci_name", "presence", "seasonal"], 
            aliases=["Scientific name", "Presence", "Seasonal"], 
            localize=True,
        ),
        style_function=lambda feature: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.5,
        },
    )
    """Function to create layer for the map"""

breeding_layer = create_folium_layer(breeding_zones, "Breeding zones", "green")
non_breeding_layer = create_folium_layer(non_breeding_zones, "Non-breeding zones", "lime")

breeding_layer.add_to(m)
non_breeding_layer.add_to(m)


### 2.4 Display of the folium map

Two options are possible : 
- display the folium map in the Notebook 
    * concentrates all results in this single notebook 
    * considerably heavier
- generate a HTML link for the folder ./results and generate a local server from the terminal to open a web page 

In [ ]:
# 1st option : Displaying the map in the notebook 
display(m)

In [13]:
# 2nd option : HTML Link 

m.save("./results/puffin_distribution_map.html")
print(f"{os.path.abspath('./results')}")

""" Then, copy cell output and enter in the bash : 
cd <absolute path you just copy> python -m http.server 8000

A new web page should open. All you have to do is select the "puffin_distribution_map" ! """

/home/onyxia/work/projet_python_2024_ENSAE/results


' Then, copy cell output and enter in the bash : \ncd <absolute path you just copy> python -m http.server 8000\n\nA new web page should open. All you have to do is select the "puffin_distribution_map" ! '

### 🧑‍💻 About local server... 
A **local server** is a web server environment that runs solely on your computer. It uses your machine as a “server” to deliver files, HTML content or web applications to a browser via a local URL (such as http://localhost:8000).
- localhost: A special address that refers to your own machine. 
- Port: A number used to distinguish different services on your machine. By default, a local server often uses port 8000 or 8080.

_Step 1: Launch a local server_
- Open a terminal and navigate to the directory containing your files.
- Run the following command:
    _python -m http.server 8000_

_Step 2: Access the server in a browser_
- Open a browser.
- Go to http://localhost:8000.
You'll see the files in the directory as if you were browsing a website.

Other informations:
- Limited access: here, the local server is only accessible from your own machine.
- Stop server: to stop it, press Ctrl+C in the terminal.
- Applications : web development (test sites or applications locally before deploying them online), data visualization (serve files such as Folium maps or interactive graphic). 

## 🌊🥵 What about climate change ? 

Let's have a look to our climate data from Copernicus. These will be used as an explanatory variable to study the evolution of puffins. 



### 3.1 MinIO Client cloud data retrieval and overview


_Nota bene : Having encountered difficulties working with files directly in S3 streams (shapefile, .nc, etc.), it is preferable to download files directly from the MinIO Client locally, to ensure optimal operation of the pandas and geopandas ecosystems, which is what the following program does._ 

In [20]:
# Creation of a subfolder to store .nc files and to download every files of the folder cds_data of the cloud 
local_cds_subfolder = os.path.join(data_dir, "local_cds_files")
os.makedirs(local_cds_subfolder, exist_ok=True)

# Extraction of files' name
cds_elements = fs.ls(f"{MY_BUCKET}/diffusion/cds_data")
cds_file_names = [os.path.basename(file) for file in cds_elements]


for file_name in cds_file_names:
    remote_path = f"{MY_BUCKET}/diffusion/cds_data/{file_name}"
    local_path = os.path.join(local_cds_subfolder, file_name)
    with fs.open(remote_path, "rb") as remote_file:
        with open(local_path, "wb") as local_file:
            local_file.write(remote_file.read())


In [40]:
import netCDF4 as nc

file_path = "/home/onyxia/work/projet_python_2024_ENSAE/data/local_cds_files/tos_Omon_HadGEM3-GC31-LL_historical_r1i1p1f3_gn_19500116-20141216.nc"

dataset = nc.Dataset(file_path, mode="r")

print("Dimensions :", dataset.dimensions.keys())
print("Variables :", dataset.variables.keys())

# Accéder à une variable spécifique
tos = dataset.variables["tos"]
print("Données TOS :", tos[:])

Dimensions : dict_keys(['time', 'bnds', 'j', 'i', 'vertices'])
Variables : dict_keys(['time', 'time_bnds', 'j', 'i', 'latitude', 'longitude', 'vertices_latitude', 'vertices_longitude', 'tos'])
Données TOS : [[[ 9.41662550e-01]]

 [[-1.27250001e-01]]

 [[ 3.79500419e-01]]

 [[ 1.38597333e+00]]

 [[ 2.79677963e+00]]

 [[ 4.25367928e+00]]

 [[ 7.98601103e+00]]

 [[ 1.02523651e+01]]

 [[ 9.84106255e+00]]

 [[ 7.91086960e+00]]

 [[ 4.66482067e+00]]

 [[ 3.60063404e-01]]

 [[-2.32054248e-01]]

 [[-5.34207337e-02]]

 [[-2.13454902e-01]]

 [[-1.32565394e-01]]

 [[ 5.72003543e-01]]

 [[ 3.17444372e+00]]

 [[ 6.62855291e+00]]

 [[ 1.17515097e+01]]

 [[ 1.16830673e+01]]

 [[ 8.28960228e+00]]

 [[ 5.25750017e+00]]

 [[ 2.94263339e+00]]

 [[ 1.38176465e+00]]

 [[ 6.43894792e-01]]

 [[-4.72491868e-02]]

 [[ 4.18912202e-01]]

 [[ 2.86302781e+00]]

 [[ 7.11993790e+00]]

 [[ 9.24420261e+00]]

 [[ 1.35426092e+01]]

 [[ 1.08062382e+01]]

 [[ 1.00197916e+01]]

 [[ 6.47607994e+00]]

 [[ 1.91733563e+00]]

 

In [ ]:
import xarray as xr

###################### VISUALIZE COPERNICUS DATA ####################

# Path to the NetCDF file
file_path = "data/data_SST.nc"

# Load the NetCDF file
data = xr.open_dataset(file_path)

# Explore the data
print(data)  # Summary of dimensions, variables, and attributes
print(data.variables)  # List of available variables

# Access a specific variable
print(data['tos'])  # Replace with an actual variable

# Extract the data from a variable as a NumPy array
variable_data = data['tos'].values

# Close the file to release resources
data.close()

import matplotlib
import matplotlib.pyplot as plt
import xarray as xr

# Assuming 'ds' is your xarray Dataset and 'tos' is the variable
# Extract the 'tos' data variable
tos = data['tos']

# Select data for a specific grid point, e.g., i=0 and j=0
tos_at_point = tos.sel(i=0, j=0, method='nearest')

# Plotting sea surface temperature through time
plt.figure(figsize=(10, 6))
tos_at_point.plot()
plt.title('Sea Surface Temperature Over Time (Grid Point: i=0, j=0)')
plt.xlabel('Time')
plt.ylabel('Sea Surface Temperature (°C)')
plt.xticks(rotation=45)
plt.tight_layout()
# Save the plot to a file (e.g., PNG)
plt.savefig('output/sst_time_series.png', dpi=300)

plt.show()

######################### VISUALIZE DIFFERENT SCENARIOS #############################

import xarray as xr
import matplotlib.pyplot as plt

# Paths to the NetCDF files
file_paths = [
    "data/tos_Omon_HadGEM3-GC31-LL_ssp126_r1i1p1f3_gn_20150116-20491216.nc",  # Scenario 1
    "data/tos_Omon_HadGEM3-GC31-LL_ssp245_r1i1p1f3_gn_20150116-20491216.nc",  # Scenario 2
    "data/tos_Omon_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_20150116-20491216.nc"   # Scenario 3
]

# Names of the scenarios for the legend
scenario_names = ["SSP1-2.6", "SSP2-4.5", "SSP5-8.5"]

# Initialize the plot
plt.figure(figsize=(12, 8))

# Loop through the files and add them to the plot
for i, file_path in enumerate(file_paths):
    # Load the NetCDF file
    data = xr.open_dataset(file_path)
    
    # Extract the sea surface temperature (tos) data
    tos = data['tos']
    
    # Select a specific grid point (e.g., i=0, j=0)
    tos_at_point = tos.sel(i=0, j=0, method='nearest')
    
    # Add the data to the plot
    plt.plot(
        tos_at_point['time'],  # X-axis: time
        tos_at_point,          # Y-axis: sea surface temperature
        label=scenario_names[i]  # Legend based on the scenario
    )
    
    # Close the NetCDF file to release resources
    data.close()

# Customize the plot
plt.title('Evolution of Sea Surface Temperature for Different Scenarios')
plt.xlabel('Time')
plt.ylabel('Sea Surface Temperature (°C)')
plt.legend(title="Scenarios", loc='upper left')  # Display the legend
plt.grid(True)
plt.xticks(rotation=45)  # Rotate the x-axis labels
plt.tight_layout()

# Save the plot
plt.savefig('output/sst_scenarios_comparison.png', dpi=300)

# Display the plot
plt.show()


####################@ VISUALIZE PUFFIN DATA #################

import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV data (puffin production rate)
puffins = pd.read_csv("data/puffin-data1_Data.csv")

# Rename the 'Year' column to 'time' to match the temperature column
puffins.rename(columns={"Year": "time", "Prod": "production_rate"}, inplace=True)

# Check the data types
print("Data types in puffins:\n", puffins.dtypes)

# Visualize the puffin production rate over the years
plt.figure(figsize=(10, 6))
plt.plot(puffins['time'], puffins['production_rate'], marker='o', linestyle='-', color='blue', label='Production Rate')
plt.title("Puffin Production Rate Over Years")
plt.xlabel("Year")
plt.ylabel("Production Rate")
plt.grid(True)
plt.legend()
plt.tight_layout()

# Save the plot (if needed)
plt.savefig('output/puffin_rate_over_years.png', dpi=300)

# Display the plot
plt.show()

##########################@@ VISUALIZE PUFFIN AND TEMPERATURE ###################
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

# Load NetCDF data (temperatures)
nc_file = "output/annual_mean_SST.nc"
data = xr.open_dataset(nc_file)

# Calculate the annual mean of temperatures
temperature = data['tos'].resample(time="1Y").mean().to_dataframe()
temperature.reset_index(inplace=True)

# Convert 'time' to full years
temperature['time'] = temperature['time'].astype(str)  # Convert cftime to string
temperature['time'] = pd.to_datetime(temperature['time'])  # Convert to datetime
temperature['time'] = temperature['time'].dt.year.astype('int64')  # Extract year as integer

# Load CSV data (puffin production rates)
puffins = pd.read_csv("data/puffin-data1_Data.csv")

# Rename the 'Year' column to 'time' to match the temperature column
puffins.rename(columns={"Year": "time", "Prod": "production_rate"}, inplace=True)

# Merge data on the 'time' column (inner join)
merged_data = pd.merge(temperature, puffins[['time', 'production_rate']], on="time", how="inner")

# Visualize temperature and production rate over the years
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot temperature on Y-axis 1
ax1.set_xlabel("Year")
ax1.set_ylabel("Sea Surface Temperature (°C)", color="tab:blue")
ax1.plot(merged_data['time'], merged_data['tos'], color="tab:blue", label="Temperature")
ax1.tick_params(axis="y", labelcolor="tab:blue")

# Add a second Y-axis for the puffin production rate
ax2 = ax1.twinx()  # Share the X-axis
ax2.set_ylabel("Puffin Production Rate", color="tab:green")
ax2.plot(merged_data['time'], merged_data['production_rate'], color="tab:green", label="Puffin Production", linestyle="--")
ax2.tick_params(axis="y", labelcolor="tab:green")

# Add a title and customize the plot
fig.suptitle("Temperature and Puffin Production Rate Over the Years", fontsize=14)
ax1.grid(True, linestyle="--", alpha=0.5)
fig.tight_layout()

# Save the plot (if necessary)
plt.savefig("output/temperature_puffin_rate_over_years.png", dpi=300)

# Display the plot
plt.show()